## Description

I wants understand why my train is different on other people where i wrong?
I train default how i train another model detection and convolution but i see in kernels in kaggle people use another techicks to train nlp model and i can't understand why and i want test it.

Links to use:
- [22nd solution](https://www.kaggle.com/c/commonlitreadabilityprize/discussion/257302)

- [gtihub link](https://github.com/kurupical/commonlit)

## default or not changed loads

In [1]:
from IPython.display import clear_output
!pip install transformers
clear_output()

In [ ]:
import re
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch import nn, optim
from torch.utils.data import  DataLoader, Dataset
import transformers

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt

from collections import defaultdict

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


path_tr = '/content/drive/MyDrive/CommonLit/input/train.csv'
path_test = '/content/drive/MyDrive/CommonLit/input/test.csv'
path_sub = '/content/drive/MyDrive/CommonLit/input/sample_submission.csv'


SEED =13
np.random.seed(SEED)
torch.manual_seed(SEED)
device = 'cuda' if torch.cuda.is_available() else 'cpu'


def make_folds(data: pd.DataFrame, split: int = 5):
    data['kfold'] = -1
    data = data.sample(frac =1).reset_index(drop=True)
    num_bins = int(np.floor(1 + np.log2(len(data))))
    data.loc[:, 'bins'] = pd.cut(
        data['target'], bins = num_bins, labels = False
    )
    kf = model_selection.StratifiedKFold(n_splits=split)
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    data = data.drop("bins", axis=1)
    return data


def clean_text(txt):
    return re.sub('[^A-Za-z]+', ' ',str(txt).lower())


df = pd.read_csv(path_tr)
df['txt'] = df['excerpt']#.apply(lambda x: clean_text(x))
df_folds = make_folds(df, 5)

In [3]:
class CL_Dataset(Dataset):

    def __init__(
        self,
        data: pd.DataFrame,
        token,
        max_len: int = 256,
        test: bool = False
        ) -> dict:
        self.data = data 
        self.max_len = max_len
        self.test = test
        self.token = token

    def __len__(self):
        return self.data.shape[0]

    def  __getitem__(self, idx: int):
        text = self.data.txt.iloc[idx]
        encode = self.token(
            text,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            add_special_tokens=True,            
            return_attention_mask=True,
            return_token_type_ids=False  
            )
        if self.test:
            target = 0
        else:
            target = self.data.target.iloc[idx]

        ids = encode["input_ids"]
        mask = encode["attention_mask"]

        return {
            "input_ids": torch.tensor(ids, dtype=torch.long),
            "attention_mask": torch.tensor(mask, dtype=torch.long),
            'target': torch.tensor(target, dtype = torch.float)  
        }

In [4]:
def loss_fn(output,target):
    return torch.sqrt(nn.MSELoss()(output,target))


def viz_curve(data: dict, title: str) -> None:
    epochs = list(range(1, EPOCH + 1))
    fig = make_subplots(
        rows=2, cols=2,
        specs=[
               [{"colspan": 2}, None],
               [{}, {}],
               ],
               vertical_spacing=0.09,
               subplot_titles=('Loss Curve',  'RMSE', 'LR')
    )
    fig.add_trace(
        go.Scatter(
            x=epochs,
            y=data['train_loss'],
            mode='lines+markers',
            name='Train Loss'),
            row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=epochs,
            y=data['valid_loss'],
            mode='lines+markers',
            name='Valid Loss'),
            row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=epochs,
            y=data['valid_rms'],
            mode='lines+markers',
            name='RMSE'),     
            row=2, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=epochs,
            y=data['lr'],
            mode='lines+markers',
            name='LR'), 
            row=2, col=2
    )
    fig.update_layout(
        height=600,
        width=600,
        showlegend=False,
        title = title,
        margin=dict(l=10, r=10, t=30, b=20),                     
        template="plotly_dark"    
    )
    fig.show()

def update_result(
    data: dict,
    predict: torch.tensor,
    target:torch.tensor
) -> None:
    loss = loss_fn(predict.squeeze(-1), target)
    data['losses'].append(loss.detach().cpu().numpy())
    data['all_pred'].append(predict.squeeze(-1).detach().cpu().numpy())
    data['all_target'].append(target.detach().cpu().numpy())
    return loss


def train(
    model:nn.Module, 
    loader: DataLoader,
    optimizer: transformers.AdamW,
    schedule: transformers.get_linear_schedule_with_warmup,
    batch: int,
    max_lenght: int
) -> list:
    model.train()
    torch.backends.cudnn.benchmark = True
    results = defaultdict(list)
    for input in loader:
        optimizer.zero_grad()
        target = input['target'].to(device)
        batch = {k:v.to(device) for k,v in input.items() if k != 'target'}   
        out = model(**batch)
        out = out.logits # BertForSequenceClassification
        loss = update_result(results, out, target)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1.0)
        optimizer.step()    
        schedule.step()
    losses = np.mean(results['losses'])
    allt = np.concatenate(results['all_target'])
    allp = np.concatenate(results['all_pred'])
    rmse = np.sqrt(mean_squared_error(allt, allp))  
    return losses, rmse


def validate(
    model:nn.Module, 
    loader: DataLoader, 
    batch: int,
    max_lenght: int
) -> list:
    model.eval()
    results = defaultdict(list)
    for input in loader:  
        target = input['target'].to(device)
        batch = {k:v.to(device) for k,v in input.items() if k != 'target'} 
        out = model(**batch)
        out = out.logits # BertForSequenceClassification
        _ = update_result(results, out, target)
    losses = np.mean(results['losses'])
    allt = np.concatenate(results['all_target'])
    allp = np.concatenate(results['all_pred'])
    rmse = np.sqrt(mean_squared_error(allt, allp)) 
    return losses, rmse


def showtime(model:nn.Module, data: pd.DataFrame, tokenizer:transformers.AutoTokenizer, fold: int) -> dict:    
    history = defaultdict(list)
    model = model.to(device)
    ttr = data[data.kfold != fold].reset_index(drop=True)
    vvl = data[data.kfold == fold].reset_index(drop=True)
    print(f'Fold: {fold + 1}, --- {ttr.shape, vvl.shape}')

    tr = CL_Dataset(ttr,tokenizer, MAX_LEN)
    vl = CL_Dataset(vvl,tokenizer, MAX_LEN)
    tr_loader = DataLoader(
        tr,
        batch_size=BATCH,
        shuffle=True
    )
    vl_loader = DataLoader(
        vl,
        batch_size=BATCH,
        shuffle=False
    )
    optimizer = transformers.AdamW(model.parameters(), lr=2e-5, correct_bias=True)
    steps = len(tr_loader) * EPOCH
    # steps = len(tr_loader)/BATCH * EPOCH
    lin_schedule = transformers.get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=1,
        num_training_steps=steps
    )
    best_rmse = np.inf
    for epoch in tqdm(range(EPOCH)):
        tr_loss, tr_rmse = train(model, tr_loader, optimizer, lin_schedule, BATCH, MAX_LEN)
        vl_loss, vl_rmse = validate(model, vl_loader, BATCH, MAX_LEN)
        lr = optimizer.param_groups[0]['lr']
        history['train_loss'].append(tr_loss)
        history['valid_loss'].append(vl_loss)
        history['valid_rms'].append(vl_rmse)
        history['lr'].append(lr)
        print(f'Epoch: {epoch}, lr: {lr}, train rmse: {tr_rmse}, vl rmse: {vl_rmse}, vl loss: {vl_loss}')
        if vl_rmse < best_rmse:
            print(f'Save rmse: {vl_rmse}')
            torch.save(model.state_dict(), f'{MODEL}_model_{fold}.pth')
            best_rmse = vl_rmse
    return history

In [5]:
class Auto_NLP_model:
    def __init__(self, name: str, n_labels: int = 1):
        print(f'Auto NLP init model : {name}')
        self.name = name
        self.n_labels=  n_labels        
    
    def loads(self):
        print('Loading configuraiton...')
        model_config = transformers.AutoConfig.from_pretrained(
                pretrained_model_name_or_path=self.name,
                num_labels=self.n_labels)
        print('Loading tokenizer...')
        tokenizer = transformers.AutoTokenizer.from_pretrained(
                pretrained_model_name_or_path=self.name)
        print('Loading model...')
        model = transformers.AutoModelForSequenceClassification.from_pretrained(
                pretrained_model_name_or_path=self.name,
                config=model_config)
        return model, tokenizer

In [6]:
"""
https://huggingface.co/transformers/pretrained_models.html
https://huggingface.co/transformers/model_doc/auto.html

"""

m = {
    # 'xlm-roberta':'xlm-roberta-base',
    # 'deberta': 'microsoft/deberta-base',
    # 'xlm':'xlm-mlm-ende-1024',
    # 'camembert': 'camembert-base',
    # 'mbart': 'facebook/mbart-large-cc25', #memory batch 4 not ends error
    'roberta': 'roberta-large'
}

In [ ]:
EPOCH = 15
BATCH = 6
MAX_LEN = 256

for k, v in m.items():
    print(k)
    model, tokenizer = Auto_NLP_model(v).loads()
    MODEL = model.__class__.__name__
    fold = 0
    title = f'Model: {k} Fold: {fold + 1}'
    history = showtime(model, df_folds, tokenizer, fold)
    viz_curve(history, title)

### rsults:

results y model:

```
# This is formatted as code
Fold: 1, --- ((2267, 8), (567, 8))
  0%|          | 0/15 [00:00<?, ?it/s]Epoch: 0, lr: 1.866995942847063e-05, train rmse: 0.771242618560791, vl rmse: 0.5320066809654236, vl loss: 0.5065193772315979
Save rmse: 0.5320066809654236
 13%|█▎        | 2/15 [13:49<1:30:08, 416.03s/it]Epoch: 1, lr: 1.7336390897865588e-05, train rmse: 0.5480484366416931, vl rmse: 0.7261704802513123, vl loss: 0.7056165933609009
 20%|██        | 3/15 [20:51<1:23:43, 418.63s/it]Epoch: 2, lr: 1.6002822367260542e-05, train rmse: 0.45481783151626587, vl rmse: 0.5933656096458435, vl loss: 0.5674742460250854
 27%|██▋       | 4/15 [27:52<1:16:56, 419.67s/it]Epoch: 3, lr: 1.4669253836655498e-05, train rmse: 0.3679790794849396, vl rmse: 0.6310817003250122, vl loss: 0.6084640622138977
 33%|███▎      | 5/15 [34:54<1:10:04, 420.41s/it]Epoch: 4, lr: 1.333568530605045e-05, train rmse: 0.3390931189060211, vl rmse: 0.7398288249969482, vl loss: 0.721051037311554
 40%|████      | 6/15 [41:56<1:03:08, 420.91s/it]Epoch: 5, lr: 1.2002116775445406e-05, train rmse: 0.2956746220588684, vl rmse: 0.5392723083496094, vl loss: 0.5154401659965515
 47%|████▋     | 7/15 [48:58<56:11, 421.40s/it]  Epoch: 6, lr: 1.066854824484036e-05, train rmse: 0.2643064260482788, vl rmse: 0.5453745126724243, vl loss: 0.5227868556976318
 53%|█████▎    | 8/15 [56:01<49:11, 421.65s/it]Epoch: 7, lr: 9.334979714235315e-06, train rmse: 0.25952309370040894, vl rmse: 0.6315166354179382, vl loss: 0.6088392734527588
 60%|██████    | 9/15 [1:03:03<42:11, 421.85s/it]Epoch: 8, lr: 8.001411183630271e-06, train rmse: 0.2210676074028015, vl rmse: 0.659906804561615, vl loss: 0.6411230564117432
 67%|██████▋   | 10/15 [1:10:05<35:10, 422.07s/it]Epoch: 9, lr: 6.667842653025225e-06, train rmse: 0.21520286798477173, vl rmse: 0.5818750262260437, vl loss: 0.5584777593612671
Epoch: 10, lr: 5.33427412242018e-06, train rmse: 0.20328344404697418, vl rmse: 0.5239043235778809, vl loss: 0.4989898204803467
Save rmse: 0.5239043235778809
 80%|████████  | 12/15 [1:24:15<21:09, 423.21s/it]Epoch: 11, lr: 4.0007055918151355e-06, train rmse: 0.18055203557014465, vl rmse: 0.5367754697799683, vl loss: 0.5130714178085327
 87%|████████▋ | 13/15 [1:31:17<14:05, 422.95s/it]Epoch: 12, lr: 2.66713706121009e-06, train rmse: 0.1665629744529724, vl rmse: 0.5970629453659058, vl loss: 0.5752689838409424
 93%|█████████▎| 14/15 [1:38:19<07:02, 422.65s/it]Epoch: 13, lr: 1.333568530605045e-06, train rmse: 0.15745727717876434, vl rmse: 0.5785849094390869, vl loss: 0.5563144683837891
100%|██████████| 15/15 [1:45:21<00:00, 421.43s/it]Epoch: 14, lr: 0.0, train rmse: 0.14649353921413422, vl rmse: 0.5729362964630127, vl loss: 0.5512558817863464

```



## adds something new

In [7]:
@dataclasses.dataclass
class Config:
    experiment_name: str
    debug: bool = False
    fold: int = 0

    nlp_model_name: str = "roberta-large"
    linear_dim: int = 128
    # dropout: float = 0
    # dropout_stack: float = 0.2
    batch_size: int = 8

    lr_bert: float = 1e-5
    lr_fc: float = 1e-4
    scheduler_params = {"num_warmup_steps": 16*100,
                        "num_training_steps": 16*3000}
    if debug:
        epochs: int = 2
    else:
        epochs: int = 15

    optimizer: Any = AdamW
    weight_decay = 0.1